![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSlqqeSaPXhlIIa3cgWd0l3TBUlzXk5rGIQZmMVheyiLF2VK001&usqp=CAU)

# *Emotion Detection Video Analysis Interactive Notebook*

Author: *Raz Friedman (r.friedman@berkeley.edu)*

Contributors: *Matthew Forbes (mforbes97@berkeley.edu), Wesley Kwong (weskwong2@ischool.berkeley.edu)*   

## Packages to be installed:

In [0]:
!pip install face_recognition

In [0]:
!pip install Pillow

In [0]:
!pip install EmoPy

In [0]:
pip install scipy==1.1.0

In [0]:
!pip install opencv-python

##Functions:

In [0]:
#run this cell

%tensorflow_version 1.x
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import warnings
import shutil
import os
import matplotlib.pyplot as plt
import re
import numpy as np
import pandas as pd
import scipy.misc
from EmoPy.src.fermodel import FERModel
from pkg_resources import resource_filename
import face_recognition
from PIL import Image
import cv2
import io
import datetime
from contextlib import redirect_stdout
from google.colab import files


def movieToPic(video_path, interval, output_path):
    '''Creates images from a video based on interval and stores them in image folder
    @movie_path: path to video that will be analyzed
    @interval: number of seconds between images to be taken from video
    @output_path: where the images will be stored'''

    video = cv2.VideoCapture(video_path)
    os.chdir(output_path) 
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = float(frame_count) / float(fps)
    duration = int(round(duration))
    fps = int(round(fps))
    frames_sec = [i * fps for i in np.arange(1,duration + 1)]
    first = frames_sec[0]
    last = frames_sec[-1]
    frames_num = np.arange(1,len(frames_sec)+1)
    frames_sec = frames_sec[1:-1]
    frames_num = frames_num[1:-1]
    frames_NEED = [x for x,y in zip(frames_sec, frames_num) if y%interval == 0]
    final = [first] + frames_NEED + [last]

    def time(count, fps):
      curr = count / fps
      return datetime.timedelta(seconds = curr)  
    
    count = 0
    success,image = video.read()
    success = True
    while success:
      success,image = video.read()
      if count in final:
        cv2.imwrite("%s.jpg" % time(count, fps), image)
      count += 1

def face_extract(image_path, out_path):
  '''Extracts faces from main image and stores them in sub-image folder.
     @image_path - path of image that will be analyzed
     @out_path - path of output saved to disk'''

  try:
    output = []
    im_name = image_path.replace("/","").replace(
        "inputs_PayAttention","").replace(
            ".jpg", "")
    path = im_name + "_faces"
    
    try:
        os.mkdir(out_path + "/" + path)
    except FileExistsError:
        pass
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image)
    for tup in face_locations:
        output.append((tup[3],tup[0],tup[1],tup[2]))

    img = Image.open(image_path)
    
    num = 1
    for i in output:
        name = "sub_" + str(num) + ".jpg"
        sub = img.crop((i[0],i[1],i[2],i[3])).resize((
            128,128)).save(out_path + '/' + path + "/" + name, optimize=True)
        num +=1
    return im_name + " " + "- Extraction Complete!"
  except:
    print("Extraction Failed")

def emote_img(path, target_emos):
  ''' Quantifies target emotions for a given image.
      @path - path of image that will be analyzed
      @target_emos - target emotions selected by user from emotion subsets ''' 
  
  sub_names = []
  model = FERModel(target_emos, verbose=False)
  for sub in os.listdir(path):
    print(sub)
    model.predict(path + "/" + sub)
    sub_names += [sub]
  return sub_names

def img_to_df(path, target_emos, out):
  ''' Converts a given image into a dataframe of analyized emotions.
      @path - path of image that will be analyzed
      @target_emos - target emotions selected by user from emotion subsets
      @out - string represention of model predictions '''

  output = "/output_PayAttention"
  df_path = output + "/dataframes"
  
  try:
    os.mkdir(df_path)
  except FileExistsError:
    pass
  
  values = []
  for emo in target_emos:
    regex = r"" + emo + ": (\d+.\d)"
    z = re.findall(regex, out)
    values += [z]

  subs_num = 0
  for i in os.listdir(path):
    subs_num += 1

  ordered = []
  for i in np.arange(subs_num):
    for j in np.arange(len(values)):
      ordered += [float(values[j][i])]
  try:
    df = pd.DataFrame(np.split(np.array(ordered), subs_num),
                    columns = target_emos)
  except ZeroDivisionError:
    df = pd.DataFrame(data = np.zeros((1,len(target_emos))), columns = target_emos)
  df = df.div(100)
  df.to_csv(df_path + "/" + path.replace(output + "/", "") 
  + "_df.csv", index = False, header=True)
  pd.option_context('display.max_rows', None, 'display.max_columns', None)
  return display(df)

def create_agg_df(path_dataframes, target_emotions):
  ''' Creates an aggregate dataframe from all dataframes.
      @path_dataframes - path to df folder
      @target_emotions - target emotions selected by user from emotion subsets
     '''
  output = "/output_PayAttention"
  vals = []
  names = []
  for i in sorted(os.listdir(path_dataframes)):
    df = pd.read_csv(path_dataframes + "/" + i)
    name = i.replace("_faces", "").replace(".csv", "")
  
    dom_emotion = ""
    curr = 0
    for col in target_emotions:
      if sum(df[col])/df.shape[0] > curr:
        dom_emotion = col
      if sum(df[col])/df.shape[0] == curr:
        dom_emotion += " & " + col

    emo_vals = np.array(df.describe().iloc[1, :]).round(3)
    full = np.append(emo_vals, dom_emotion)
    names += [name]
    vals += [full]

  agg = pd.DataFrame(vals,
                     columns = np.append(target_emotions, "Dominant Emotion"),
                     index = names)
  agg.to_csv(path_dataframes + "/" + "aggregate.csv", index = True, header=True)
  pd.option_context('display.max_rows', None, 'display.max_columns', None)
  display(agg)

def EDA(path_dataframes, target_emotions):
  ''' Performs exploratory data analysis on given dataframes.
      @path_dataframes - path to df folder
      @target_emotions - target emotions selected by user from emotion subsets
      '''
  output = "/output_PayAttention"
  vis = output + "/data_visualizations"
  try:
    os.mkdir(vis)
  except FileExistsError:
    pass

  agg = pd.read_csv(path_dataframes + "/aggregate.csv")
  agg = agg.rename(columns={'Unnamed: 0': 'Image'})
  agg = agg.set_index("Image")
  no_dom = agg.drop(columns=['Dominant Emotion'])
  dom = agg["Dominant Emotion"].to_frame(0).groupby(0)[0].count()
  for emo in target_emotions:
    if emo not in dom.index:
      s = pd.Series([0])
      dom = dom.append(s)
      dom = dom.rename({0: emo})
  dom = dom.to_frame()
  
  print()
  for emo in target_emotions:
    print(" ** For emotion " + emo 
        + " the max value of " + str(agg[emo].max()) 
        + " is in image " + agg[emo].idxmax().replace("_df", ""))
    print()
  
  no_dom.boxplot(figsize=(10,8), fontsize=15)
  plt.title("Boxplot - " + str(target_emotions), fontsize=16)
  plt.savefig(vis+ "/Boxplot")

  no_dom.plot.barh(figsize=(10,8), fontsize=15)
  plt.title(str(target_emotions) + " Per Image", fontsize=16)
  plt.ylabel("Images", fontsize=16)
  plt.savefig(vis + "/Barh_Per_Emotion")

  no_dom.plot.line(figsize=(10,8), grid = True)
  plt.title("Changes In " + str(target_emotions) + " Over Time", fontsize=16)
  plt.xlabel("Images Over Time", fontsize=16)
  plt.savefig(vis + "/Linegraph_emotions_over_time")

  dom.plot.bar(figsize=(10,8), fontsize=16, legend = False, rot = 0)
  plt.xlabel("Emotions", fontsize=16)
  plt.title("Dominant Emotion Count", fontsize=16)
  plt.savefig(vis + "/Bar_Dom_Emotion")

def helper_output_emo(emo_subset):
  ''' Helper function for pipline.
      @emo_subset - target emotions selected by user from emotion subsets
      '''
  f = io.StringIO()
  with redirect_stdout(f):
    for i in sorted(os.listdir('/output_PayAttention')):
      emote_img("/output_PayAttention" + "/" + i, emo_subset)
  out = f.getvalue()
  return out

def pipeline(vid_path, vid_interval, emo_subset):
  ''' Function that runs through entire emotion analysis pipeline.
      @vid_path - path to video
      @vid_interval - analysis interval(seconds) duration
      @emo_subset - target emotions selected by user from emotion subsets
     '''
  inputs = "/inputs_PayAttention"
  output = "/output_PayAttention"
  os.mkdir(inputs)
  os.mkdir(output)
  try:
    movieToPic(vid_path, vid_interval, inputs)
  except:
    pass
  if len(os.listdir(inputs + '/') ) == 0:
    return "Directory is empty - Please upload video"
  for i in sorted(os.listdir(inputs)):
    print(face_extract(inputs + '/' + i, output))
  print()
  print("**********************************************")
  print()
  print()
  out = helper_output_emo(emo_subset)
  path_list = []
  for i in sorted(os.listdir(output)):
    path_list += [i]

  emo_vals = out.split('Initializing')
  emo_vals.pop(0)
  emo_vals
  for tup in zip(path_list, emo_vals):
    print(tup[0])
    print("-------------------------------")
    img_to_df(output + "/" + tup[0], emo_subset, tup[1])
    print()
  print("Aggregate DataFrame")
  print("================================")
  print()
  create_agg_df(output + "/dataframes", emo_subset)
  EDA(output + "/dataframes", emo_subset)

  # DELETES ALL INPUTS & OUTPUTS
  shutil.rmtree('/output_PayAttention')
  shutil.rmtree('/inputs_PayAttention')


## Notebook UI

*User*, please upload video.mp4 for analysis

In [0]:
vids = files.upload()
display(vids)

*User*, please select interval (in seconds) by which video will be analyized

*Example: 2 = video analysis performed every 2 seconds*

In [0]:
import ipywidgets as widgets
from IPython.display import display

inter = widgets.IntText(
    value=1,
    description='Interval:',
    disabled=False
)

display(inter)

*User*, please select emotion subset for analysis

In [0]:
emo = widgets.RadioButtons(
    options=['anger,fear,surprise,calm', 'disgust,surprise,happiness',
             'anger,fear,surprise', "anger,fear,calm", "anger,happiness",
             "anger,happiness,calm", "disgust,anger,fear",
             "disgust,surprise,calm", "disgust,sadness,surprise"],
    layout={'width': 'max-content'},
    description='Emotions:',
    disabled=False
)

display(emo)

### *Run this cell for results:*

In [0]:
for vid in vids.keys():
  pipeline(os.path.abspath(vid), inter.value, emo.value.split(","))